# Machine Learning Trading Bot

In this Challenge, you’ll assume the role of a financial advisor at one of the top five financial advisory firms in the world. Your firm constantly competes with the other major firms to manage and automatically trade assets in a highly dynamic environment. In recent years, your firm has heavily profited by using computer algorithms that can buy and sell faster than human traders.

The speed of these transactions gave your firm a competitive advantage early on. But, people still need to specifically program these systems, which limits their ability to adapt to new data. You’re thus planning to improve the existing algorithmic trading systems and maintain the firm’s competitive advantage in the market. To do so, you’ll enhance the existing trading signals with machine learning algorithms that can adapt to new data.

## Instructions:

Use the starter code file to complete the steps that the instructions outline. The steps for this Challenge are divided into the following sections:

* Establish a Baseline Performance

* **Tune the Baseline Trading Algorithm**

* Evaluate a New Machine Learning Classifier

* **Create an Evaluation Report**

#### Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 

1. Import the OHLCV dataset into a Pandas DataFrame.

2. Generate trading signals using short- and long-window SMA values. 

3. Split the data into training and testing datasets.

4. Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

5. Review the classification report associated with the `SVC` model predictions. 

6. Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

7. Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

8. Write your conclusions about the performance of the baseline trading algorithm in the `README.md` file that’s associated with your GitHub repository. Support your findings by using the PNG image that you saved in the previous step.

#### Tune the Baseline Trading Algorithm

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. (You’ll choose the best by comparing the cumulative products of the strategy returns.) To do so, complete the following steps:

1. Tune the training algorithm by adjusting the size of the training dataset. To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing the training window?

> **Hint** To adjust the size of the training dataset, you can use a different `DateOffset` value&mdash;for example, six months. Be aware that changing the size of the training dataset also affects the size of the testing dataset.

2. Tune the trading algorithm by adjusting the SMA input features. Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

3. Choose the set of parameters that best improved the trading algorithm returns. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

#### Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. To do so, complete the following steps:

1. Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

2. Using the original training data as the baseline model, fit another model with the new classifier.

3. Backtest the new model to evaluate its performance. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. Answer the following questions: Did this new model perform better or worse than the provided baseline model? Did this new model perform better or worse than your tuned trading algorithm?

#### Create an Evaluation Report

In the previous sections, you updated your `README.md` file with your conclusions. To accomplish this section, you need to add a summary evaluation report at the end of the `README.md` file. For this report, express your final conclusions and analysis. Support your findings by using the PNG images that you created.


In [ ]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

---

## Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 


### Step 1: mport the OHLCV dataset into a Pandas DataFrame.

In [ ]:
# Import the OHLCV dataset into a Pandas Dataframe
ohlcv_df = pd.read_csv(
    Path("./Resources/emerging_markets_ohlcv.csv"), 
    index_col='date', 
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
print(ohlcv_df.head())
print(ohlcv_df.tail())

In [ ]:
ohlcv_df.shape

In [ ]:
# Filter the date index and close columns
signals_df = ohlcv_df.loc[:, ["close"]]

# Use the pct_change function to generate  returns from close prices
signals_df["Actual Returns"] = signals_df["close"].pct_change()

# Drop all NaN values from the DataFrame
signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

## Step 2: Generate trading signals using short- and long-window SMA values. 

In [ ]:
# Set the short window and long window
short_window = 4
long_window = 100

# Generate the fast and slow simple moving averages (4 and 100 days, respectively)
signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Initialize the new Signal column
signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
signals_df['Signal'].value_counts()

In [ ]:
# Calculate the strategy returns and add them to the signals_df DataFrame
# Observe that the signal considered is the one from the previous period, 
# because that is the prediction we would like to have given
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy Returns']).cumprod().plot()

### Step 3: Split the data into training and testing datasets.

In [ ]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

# Review the DataFrame
X.head()

In [ ]:
# Create the target set selecting the Signal column and assigning it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

In [ ]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

In [ ]:
len(X_train)

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_train.head()

In [ ]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape)
print(y_train.shape)

### Step 4: Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

In [ ]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred


### Step 5: Review the classification report associated with the `SVC` model predictions. 

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)


### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_model.predict(X_test_scaled)
predictions_df['Predicted'].value_counts()

In [ ]:
# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = predictions_df['Predicted'] * predictions_df['Actual Returns'] 

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

In [ ]:
cum_actual=(1+predictions_df['Actual Returns']).cumprod()-1
cum_pred=(1+predictions_df['Strategy Returns']).cumprod()-1
print(cum_actual[-1])
print(cum_pred[-1])

In [ ]:
predictions_df['Predicted'].value_counts()

In [ ]:
comparison=pd.concat([y,predictions_df['Predicted']], axis=1)
comparison[comparison['Signal']!=comparison['Predicted']]
comparison

### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [ ]:
# Plot the actual returns versus the strategy returns
predictions_df['Original Cummulative Return']=(1+predictions_df['Actual Returns']).cumprod()
predictions_df['Machine Learning Strategy Cummulative Return']=(1+predictions_df['Strategy Returns']).cumprod()

predictions_df.hvplot.line(
    y=['Original Cummulative Return', 'Machine Learning Strategy Cummulative Return'],
    title='Comparative: Cummultive Returns for the Technical Indicator Strategy and the Machine Learning Strategy',
    width = 900,
    legend='bottom'
    
)


---

## Tune the Baseline Trading Algorithm

## Step 6: Use an Alternative ML Model and Evaluate Strategy Returns

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. You’ll choose the best by comparing the cumulative products of the strategy returns.

### Step 1: Tune the training algorithm by adjusting the size of the training dataset. 

To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing the training window?

### Step 2: Tune the trading algorithm by adjusting the SMA input features. 

Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

### Step 3: Choose the set of parameters that best improved the trading algorithm returns. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

In [27]:
# Step 1: Slice data into different periods



In [41]:
results_slicing=[]

for i in range(1,55):
# Select the start of the training period
    training_begin = X.index.min()

    # Display the training begin date
    #print(training_begin)

    # Select the ending period for the training data with an offset of 3 months
    training_end = X.index.min() + DateOffset(months=i)

    # Display the training end date
    #print(training_end)

    # Generate the X_train and y_train DataFrames
    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]

    # Generate the X_test and y_test DataFrames
    X_test = X.loc[training_end+DateOffset(hours=1):]
    y_test = y.loc[training_end+DateOffset(hours=1):]


    # Scale the features DataFrames
    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Apply the scaler model to fit the X-train data
    X_scaler = scaler.fit(X_train)

    # Transform the X_train and X_test DataFrames using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # From SVM, instantiate SVC classifier model instance
    svm_model = svm.SVC()

    # Fit the model to the data using the training data
    svm_model = svm_model.fit(X_train_scaled, y_train)

    # Use the testing data to make the model predictions
    svm_pred = svm_model.predict(X_test_scaled)
    
    

    # Create a predictions DataFrame
    predictions_df = pd.DataFrame(index=X_test.index)

    # Add the SVM model predictions to the DataFrame
    predictions_df['Predicted'] = svm_model.predict(X_test_scaled)

    # Add the actual returns to the DataFrame
    predictions_df['Actual Returns'] = signals_df['Actual Returns']

    # Add the strategy returns to the DataFrame
    predictions_df['Strategy Returns'] = predictions_df['Predicted'] * predictions_df['Actual Returns'] 
    
    
    
    # ROI in the testing window
    ROI_actual=(1+predictions_df['Actual Returns']).cumprod()-1
    ROI_pred=(1+predictions_df['Strategy Returns']).cumprod()-1

    # Review the model's predicted values
    # svm_pred
    # print(len(svm_pred))
    # print(y_test.shape)
    results_slicing.append(
        {'Months': i, 
         'Accuracy':accuracy_score(y_test, svm_pred), 
         'ROI Predicted (%)':ROI_pred[-1]*100, 
         'ROI Actual (%)':ROI_actual[-1]*100,
         
    })

2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00
2015-04-02 15:00:00


KeyboardInterrupt: 

In [29]:
results_slicing_df=pd.DataFrame(results_slicing)
results_slicing_df.sort_values(by='ROI Predicted (%)', ascending=False, inplace=True)

print(results_slicing_df.head())

    Months  Accuracy  ROI Predicted (%)  ROI Actual (%)
19      20  0.565530          88.405812       57.884797
18      19  0.564829          88.305666       54.172430
5        6  0.558204          84.166422       56.042930
6        7  0.555641          81.002770       51.861944
9       10  0.561974          70.812392       78.632479


In [30]:
#Plot of results for 
results_to_plot_sliciing_df=results_slicing_df.sort_values(by='Months')

plot_accuracy=results_to_plot_sliciing_df.hvplot(
    x='Months', 
    y=['Accuracy'],
    ylim=[0,1],
    title="Accuracy versus Length of the Training Window")

plot_performance=results_to_plot_sliciing_df.hvplot(
    x='Months', 
    y=['ROI Predicted'],
    title="Predicted ROI versus Length of the Training Window")

plot_accuracy+plot_performance

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['ROI Predicted']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

### Tuned model selected with split window = 20

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
#print(training_begin)

# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=20)

# Display the training end date
#print(training_end)

# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]


# Scale the features DataFrames
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)


# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred, zero_division=0)

# Print the classification report
print(svm_testing_report)

# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)
print(predictions_df.shape)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_model.predict(X_test_scaled)

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = predictions_df['Predicted'] * predictions_df['Actual Returns'] 

# Review the DataFrame
print(predictions_df['Predicted'].value_counts())

# Plot the actual returns versus the strategy returns
predictions_df['Original Cummulative Return']=(1+predictions_df['Actual Returns']).cumprod()
predictions_df['Machine Learning Strategy Cummulative Return']=(1+predictions_df['Strategy Returns']).cumprod()

predictions_df.hvplot.line(
    y=['Original Cummulative Return', 'Machine Learning Strategy Cummulative Return'],
    title='Tuned Model Comparison: Cummultive Returns for the Actual Returns and the Tuned SVM Machine Learning Strategy Return',
    width = 900,
    legend='bottom'
    
)

**Answer the following question:** What impact resulted from increasing or decreasing the training window?

There is not a significant impact in terms of accuracy whe selecting different sizes of windows. The SVM tends to pred¡ct to go long most of the time at any size.
There is an impact on the performance, though. I would not select the window size just based on performance, but since the accuracy does not change much, I would select window=20 since at that size we get the maximum ROI, and the same accuracy than other windows.

In [49]:
# Step 2: adjust SMA windows


window_size_for_split=20
results_sma=[]
# Adjust the short window of the SMA

for long_window_size in range(30,200, 10):
    for short_window_size in range(2,(long_window_size-5), 5):

        # Resetting initial data (This was added because the beggining date was being changed during the loop)
        signals_df = ohlcv_df.loc[:, ["close"]]

        # Use the pct_change function to generate  returns from close prices
        signals_df["Actual Returns"] = signals_df["close"].pct_change()

        # Drop all NaN values from the DataFrame
        signals_df = signals_df.dropna()


        # Set the short window and long window
        short_window = short_window_size
        long_window = long_window_size

        # Generate the fast and slow simple moving averages (4 and 100 days, respectively)
        signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
        signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

        signals_df = signals_df.dropna()

        # Initialize the new Signal column
        signals_df['Signal'] = 0.0

        # When Actual Returns are greater than or equal to 0, generate signal to buy stock long
        signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

        # When Actual Returns are less than 0, generate signal to sell stock short
        signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

        # Calculate the strategy returns and add them to the signals_df DataFrame
        # Observe that the signal considered is the one from the previous period, 
        # because that is the prediction we would like to have given
        signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()


        # Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
        X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

        # Create the target set selecting the Signal column and assigning it to y
        y = signals_df['Signal']


        # Select the start & end of the training period
        training_begin = X.index.min()
        training_end = X.index.min() + DateOffset(months=window_size_for_split)


        # Generate the X_train and y_train DataFrames
        X_train = X.loc[training_begin:training_end]
        y_train = y.loc[training_begin:training_end]

        # Generate the X_test and y_test DataFrames
        X_test = X.loc[training_end+DateOffset(hours=1):]
        y_test = y.loc[training_end+DateOffset(hours=1):]


        # Scale the features DataFrames

        # Create a StandardScaler instance
        scaler = StandardScaler()

        # Apply the scaler model to fit the X_train data
        X_scaler = scaler.fit(X_train)

        # Transform the X_train and X_test DataFrames using the X_scaler
        X_train_scaled = X_scaler.transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)

        ### Step 4: Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

        # From SVM, instantiate SVC classifier model instance, fit the model and predict
        svm_model = svm.SVC()
        svm_model = svm_model.fit(X_train_scaled, y_train)
        svm_pred = svm_model.predict(X_test_scaled)


        # Create a predictions DataFrame
        predictions_df = pd.DataFrame(index=X_test.index)

        # Add the SVM model predictions, Actual, and Strategy Returns to the DataFrame
        predictions_df['Predicted'] = svm_model.predict(X_test_scaled)
        predictions_df['Actual Returns'] = signals_df['Actual Returns']
        predictions_df['Strategy Returns'] = predictions_df['Predicted'] * predictions_df['Actual Returns']  

        # ROI in the testing window
        ROI_actual=(1+predictions_df['Actual Returns']).cumprod()-1
        ROI_pred=(1+predictions_df['Strategy Returns']).cumprod()-1

        # Review the model's predicted values
        # svm_pred
        # print(len(svm_pred))
        # print(y_test.shape)
        new_row={'Months': window_size_for_split,
             'SMA Short':short_window,
             'SMA Long':long_window,
             'Accuracy':accuracy_score(y_test, svm_pred), 
             'ROI Pred (%)':ROI_pred[-1]*100, 
             'ROI Actual (%)':ROI_actual[-1]*100,
        }
        results_sma.append(new_row)
        #print(new_row)

In [97]:


#print(results_sma_df.head(20))


sma_performance_heatmap=results_sma_df.hvplot.heatmap(
    x='SMA Long', 
    y='SMA Short', 
    xlabel='SMA Long Size Window [days]',
    ylabel='SMA Short Size Window [days]',
    C='ROI Pred (%)', 
    colorbar=True,
    width=500,
    title='ML Model Performance (ROI%) versus SMA Size Window'
) 

sma_accuracy_heatmap=results_sma_df.hvplot.heatmap(
    x='SMA Long', 
    y='SMA Short', 
    xlabel='SMA Long Size Window [days]',
    ylabel='SMA Short Size Window [days]',
    C='Accuracy', 
    colorbar=True,
    width=500,
    title='ML Model Accuracy Score versus SMA Size Window'
)


sma_performance_heatmap + sma_accuracy_heatmap

:Layout
   .HeatMap.I  :HeatMap   [SMA Long,SMA Short]   (ROI Pred (%))
   .HeatMap.II :HeatMap   [SMA Long,SMA Short]   (Accuracy)

In [98]:
# Additional considerations to choose the optimal combination
# Created a balance measurement called 'accuracy and performace rate' to give more relevance to the windows that generated 
# higher accuracy together with higher performance. This rate is just an additional guidance.

results_sma_df=pd.DataFrame(results_sma)
results_sma_df['accuracy and performance rate']=results_sma_df['Accuracy']*results_sma_df['ROI Pred (%)']/results_sma_df['ROI Actual (%)']

# Visualizing the 'accuracy and performance rate' 
sma_accuracy_adjusted_heatmap=results_sma_df.hvplot.heatmap(
    x='SMA Long', 
    y='SMA Short', 
    C='accuracy and performance rate', 
    colorbar=True,
    width=1000,
    title='ML Model Accuracy and Performance Rate versus SMA Size Window'
)
sma_accuracy_adjusted_heatmap

:HeatMap   [SMA Long,SMA Short]   (accuracy and performance rate)

In [100]:
# Display the results of the iteration
# Order set up by the the 'accuracy and performance rate'
results_sma_df.sort_values(by='accuracy and performance rate', ascending=False, inplace=True)
results_sma_df.head(10)

,Months,SMA Short,SMA Long,Accuracy,ROI Pred (%),ROI Actual (%),accuracy and performance rate
140,21,2,130,0.564724,85.643722,53.888633,0.897501
105,21,47,110,0.565884,87.700638,56.921633,0.871872
325,21,27,190,0.559577,77.838721,50.089767,0.869573
155,21,77,130,0.564094,82.427333,53.888633,0.862831
291,21,32,180,0.561332,79.072365,52.380952,0.847366
96,21,2,110,0.564632,84.055590,56.921633,0.833787
221,21,2,160,0.558541,86.110029,58.784425,0.818176
117,21,2,120,0.563110,75.908989,53.817847,0.794255
176,21,57,140,0.558749,75.123247,54.385965,0.771799
126,21,47,120,0.564054,72.687669,53.817847,0.761825


**Answer the following question:** What impact resulted from increasing or decreasing either or both of the SMA windows?

We see a similar pattern in terms of the accuracy and profitability. 

* **Accuracy:** we see that small windows generate more consistent values and higher accuracy. Also, we see that the short window size became more relevant when the long window surpass 110 days. For those cases, very short term windows generate better accuracy. The range of change of the accuracy is very low, as can be seen in the color scale, which ranges from 0.535 to 0.565. The good news is that always the values are above 50%, which means, the model is better than flipping a coin. Still we would like to be better, though.

* **Performance:** we see a similar pattern than for accuracy in terms of the relevance of the windows sizes. However, the differences in performance are much more notorious, when long term windows above 150 days give very low performance when combined with short windows of about half the size of the long window. It makes sense that considering too much of the past it may be not profitable for trading strategies. Long term windows are usually used for allocation for long term investments.

The selected combination of the windows is the second in the list, because it has a higher accuracy than the first, and a more balance size of the windows than the first:

    > Long Window:130 days
    
    > Short: 77 days
    
    > Accuracy: 0.566
    
    > Performace: 87.7%


---

## Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. 

### Step 1:  Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

In [ ]:
# Import a new classifier from SKLearn
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Initiate the model instance
adaboost_model=AdaBoostClassifier(n_estimators=50)
adaboost_model

### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

In [ ]:
# Fit the model using the training data
model = adaboost_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
pred_adaboost = adaboost_model.predict(X_test_scaled)

# Review the model's predicted values
pred_adaboost


### Step 3: Backtest the new model to evaluate its performance. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. 

Answer the following questions: 
Did this new model perform better or worse than the provided baseline model? 
Did this new model perform better or worse than your tuned trading algorithm?

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
adaboost_report=classification_report(y_test, pred_adaboost)

# Print the classification report
print("         AdaBoost Classification Report")
print(adaboost_report)

print(f"\n    Support Vector Machine Classification Report")
print(svm_testing_report)


In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
predictions_adaboost_df=pd.DataFrame(index=X_test.index)

# Add the AdaBoost model predictions to the DataFrame
predictions_adaboost_df['Predictions AdaBoost']=pred_adaboost

# Add the actual returns to the DataFrame
predictions_adaboost_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_adaboost_df['AdaBoost Returns'] = predictions_adaboost_df['Predictions AdaBoost']* predictions_adaboost_df['Actual Returns'] 

# Review the DataFrame
predictions_adaboost_df.head()



In [ ]:
# Plot the actual returns versus the strategy returns
predictions_adaboost_df.hvplot(
        y=['Actual Returns','AdaBoost Returns'],
        width=1000,
        title='Comparative: Returns for the Technical Indicator Strategy versus AdaBoost ML Strategy',
        legend='bottom'
)


In [ ]:
predictions_adaboost_df['Actual Returns'].describe()
#,'AdaBoost Returns']

In [ ]:
# Plot cummulative returns
predictions_adaboost_df['Technical Indicator Cummulative Return']=(1+predictions_adaboost_df['Actual Returns']).cumprod()
predictions_adaboost_df['AdaBoost Cummulative Return']=(1+predictions_adaboost_df['AdaBoost Returns']).cumprod()

predictions_adaboost_df.hvplot(
        y=['Technical Indicator Cummulative Return','AdaBoost Cummulative Return'],
        width=800,
        title='Comparative: Cummulative Return of Actual Returns versus Cummulative Return on AdaBoost ML Strategy',
        legend='bottom'
)

In [ ]:
# Plot comparative of the three models

predictions_adaboost_df[['AdaBoost Returns','AdaBoost Cummulative Return']][10:30]